In [ ]:
import os
import shutil
from typing import List, Dict

# Script for renaming and moving solution files from tester_results to data

In [ ]:
source_path = "/Users/petr/Documents/Projects/Python/AlgorithmTester/analysis/tester_results/DataAnalysis"
dest_path = "/Users/petr/Documents/Projects/Python/AlgorithmTester/data/DataAnalysis"

dry_run: bool = True

for root, dirs, files in os.walk(source_path):
    print(f"{root}")
    dir_name: str = root.split("/")[-1]
    for filename in files:
        if "column" in filename or "DPWeight" not in filename:
            continue
                   
        split = filename.split("_")
        split = [split[0], split[1], split[4]]
        new_name: str = '_'.join(split)
            
        print(f'{dest_path}/{dir_name}/{split[1]}/{new_name}')
            
        if not dry_run:
            os.rename(f'{root}/{filename}', f'{dest_path}/{dir_name}/{split[1]}/{new_name}')

# Read lines of file in batches

In [ ]:
from itertools import islice

filepath: str = '../data/UnitTest/NK10_inst.dat'

# How many lines are read at once    
batch_size: int = 50
    
with open(filepath, "r") as file:
    batch: List[str] = [x.strip() for x in islice(file, batch_size)]
    while len(batch) > 0:
        for instance in batch:
            print(instance)
        
        batch = [x.strip() for x in islice(file, batch_size)]

In [ ]:
from package_parsers.knapsack_parser import KnapsackParser

def get_batch(parser, file, batch_size):
    batch = list()
    cntr: int = 0
    parsed_data = parser.get_next_instance(file)
    
    while (parsed_data is not None) and cntr < batch_size:
        batch.append(parsed_data)

        cntr += 1
        parsed_data = parser.get_next_instance(file)
        
    return batch

parser: KnapsackParser = KnapsackParser()
filepath: str = '../data/UnitTest/NK10_inst.dat'

# How many lines are read at once    
batch_size: int = 5
    
with open(filepath, "r") as file:
    batch = get_batch(parser, file, batch_size)
    
    while len(batch) > 0:
        print(batch)

# yield test

In [ ]:
def get_instance(cnt: int):
    for i in range(cnt):
        yield i
    
algorithms = ["A", "B", "C"]

for alg in algorithms:
    it = get_instance(5)
    
    for res in it:
        print(f'{alg}:{res}')
        
    print("---------------")

# Dummy ConcurrencyStrategy

In [ ]:
import multiprocessing
import concurrent.futures
import time

# Common functions

def get_res(algorithm: str, instance: Dict[str, object]):
    print(f"Getting res for {instance} using {algorithm}.")
    time.sleep(1)
    return f'Ran {algorithm} for {instance}.'

def get_instance(file: str):
    output = dict()
    
    for i in range(2):
        output = {"input": file, "instance": i}
        yield output
    
def run_tester_for_file(filename: str):
    for algorithm in ["A", "B"]:
        it = get_instance(f'{filename}_opened')
        
        for instance in it:
            res = get_res(algorithm, instance)
            print(f"Writing (\'{res}\') to results file for input file {filename}.")

            
# Runners
filenames = [f'File{i}' for i in range(2)]


class BaseRunner:
    
    def start(self):
        for filename in filenames:
            run_tester_for_file(filename)

class ConcurrentFilesRunner:
    
    def start(self):
        with concurrent.futures.ProcessPoolExecutor() as executor:
            batch = filenames
            futures = [executor.submit(run_tester_for_file, filename) for filename in batch]
            
class ConcurrentInstancesRunner:
    
    def start(self):
        with concurrent.futures.ProcessPoolExecutor() as executor:
            batch = filenames
            
            #concurrent.future.as_completed(futures)
            for filename in filenames:
                for algorithm in ["A", "B"]:
                    batch = [instance for instance in get_instance(f'{filename}_opened')]
                    futures = [executor.submit(get_res, instance, algorithm) for instance in batch]

                    for future in concurrent.futures.as_completed(futures):
                        res = future.result()
                        print(f"Writing (\'{res}\') to results file for input file {filename}.")

In [ ]:
# Test BaseRunner
start = time.perf_counter()

simple_runner = BaseRunner()
simple_runner.start()

finish = time.perf_counter()
print(f'Finished task in {round(finish - start, 2)} second(s)')

In [ ]:
# Test ConcurrentFilesRunner
start = time.perf_counter()

simple_runner = ConcurrentFilesRunner()
simple_runner.start()

finish = time.perf_counter()
print(f'Finished task in {round(finish - start, 2)} second(s)')

In [ ]:
# Test ConcurrentInstancesRunner
start = time.perf_counter()

simple_runner = ConcurrentInstancesRunner()
simple_runner.start()

finish = time.perf_counter()
print(f'Finished task in {round(finish - start, 2)} second(s)')